## ATTENTION

Ce Notebook nécessite des ressources GPU. 

<table class="tfo-notebook-buttons" align="left">
  <td>
    <a target="_blank" href="https://colab.research.google.com/github/lsteffenel/ED-SNI-IntroDL/blob/main/04_Utiliser_un_modele_existant.ipynb">
    <img src="https://www.tensorflow.org/images/colab_logo_32px.png" />
    Run in Google Colab</a>
  </td>
</table>

# Utiliser des modèles pré-entraînés

Bien qu'il soit souvent nécessaire de disposer d'un grand ensemble de données bien annotées pour résoudre un défi d'apprentissage profond, il existe de nombreux modèles pré-entraînés disponibles gratuitement que nous pouvons télécharger et utiliser directement. 

Lorsque vous décidez de vous lancer dans votre propre projet d'apprentissage profond, c'est une bonne idée de commencer par rechercher des modèles existants en ligne qui peuvent vous aider à atteindre votre objectif.

Plusieurs répositoires disponibilisent des modèles pré-entraînés, comme par exemple TensorFlow Hub et [NGC](https://ngc.nvidia.com/catalog/models). Github contient aussi plusieurs projets qui rendent leurs modèles disponibles. 

## Objectifs

* Utiliser Keras pour charger un modèle pré-entraîné
* Prétraiter vos propres images pour travailler avec le modèle pré-entraîné
* Utilisez le modèle pré-entraîné pour effectuer une inférence précise sur vos propres images.

## Une porte automatisée pour chiens

Dans cette section, nous allons créer une porte pour chiens qui ne laisse entrer et sortir que les chiens (et non les autres animaux). Ainsi, nous pouvons garder nos chats à l'intérieur et les autres animaux à l'extérieur, là où ils doivent être. En utilisant les techniques abordées jusqu'à présent, nous aurions besoin d'un très grand ensemble de données contenant des photos de nombreux chiens, ainsi que d'autres animaux. Heureusement, il existe un modèle facilement disponible qui a été entraîné sur un vaste ensemble de données comprenant de nombreux animaux. 

Le [défi ImageNet] (https://en.wikipedia.org/wiki/ImageNet#History_of_the_ImageNet_challenge) a produit de nombreux modèles de pointe pouvant être utilisés pour la classification d'images. Ils ont été entraînés sur des millions d'images et peuvent classer avec précision des images dans 1 000 catégories différentes. Un grand nombre de ces catégories sont des animaux, y compris des races de chiens et de chats. Il s'agit d'un modèle parfait pour notre porte pour chien.


## Télécharger le modèle pré-entraîné

Nous commencerons par télécharger le modèle. Les modèles ImageNet entraînés peuvent être téléchargés directement dans la bibliothèque Keras. Vous pouvez consulter les modèles disponibles et leurs détails [ici] (https://keras.io/api/applications/#available-models). N'importe lequel de ces modèles peut être utilisé pour notre exercice. 

Nous choisirons un modèle couramment utilisé appelé [VGG16](https://keras.io/api/applications/vgg/) :

In [ ]:
from tensorflow.keras.applications import VGG16
  
# load the VGG16 network *pre-trained* on the ImageNet dataset
model = VGG16(weights="imagenet")

Maintenant qu'il est chargé, jetons un coup d'œil au modèle. Il ressemble beaucoup à notre modèle convolutif de l'exercice sur le langage des signes. Faites attention à la première couche (la couche d'entrée) et à la dernière couche (la couche de sortie). Comme pour les exercices précédents, nous devons nous assurer que nos images correspondent aux dimensions d'entrée attendues par le modèle. Il est également utile de comprendre ce que le modèle renverra de la couche de sortie finale.

In [ ]:
model.summary()

### Dimensions d'entrée
Nous pouvons voir que le modèle attend des images de la forme (224, 224, 3) correspondant à 224 pixels de haut, 224 pixels de large et 3 canaux de couleur. Comme nous l'avons appris dans notre dernier exercice, les modèles Keras peuvent accepter plus d'une image à la fois pour la prédiction. Si nous ne transmettons qu'une seule image, la forme sera (1, 224, 224, 3). Nous devrons nous assurer que les images que nous transmettons à notre modèle pour la prédiction correspondent à ces dimensions. 

### Dimensions de sortie
Nous pouvons également constater que le modèle renvoie une prédiction de la forme 1000. Rappelez-vous que dans notre premier exercice (MNIST), la forme de sortie de notre modèle était 10, correspondant aux 10 chiffres différents. 
Ici, nous avons 1000 catégories possibles dans lesquelles l'image sera placée. Bien que l'ensemble des données ImageNet comporte plus de 20 000 catégories, le concours et les modèles pré-entraînés qui en résultent n'utilisent qu'un sous-ensemble de 1000 de ces catégories. Nous pouvons jeter un coup d'œil à toutes ces [catégories possibles] (https://gist.github.com/yrevar/942d3a0ac09ec9e5eb3a). 

Un grand nombre de ces catégories sont des animaux, y compris de nombreux types de chiens et de chats. Les chiens sont les catégories 151 à 268. Les chats sont les catégories 281 à 285. Nous pourrons utiliser ces catégories pour indiquer à notre porte pour chien quel type d'animal se trouve à notre porte et si nous devons le laisser entrer ou non.


## Chargement d'une image
Nous allons commencer par charger une image et l'afficher, comme nous l'avons fait dans les exercices précédents.

In [ ]:
!wget http://urca.lsteffenel.fr/IntroDL/data.tar.gz -O data.tar.gz
!tar -xvzf data.tar.gz


In [ ]:
import matplotlib.pyplot as plt
import matplotlib.image as mpimg

def show_image(image_path):
    image = mpimg.imread(image_path)
    print(image.shape)
    plt.imshow(image)

In [ ]:
show_image("data/doggy_door_images/happy_dog.jpg")

## Preprocessement de l'image

Ensuite, nous devons prétraiter l'image pour qu'elle soit prête à être envoyée dans le modèle. C'est exactement ce que nous avons fait dans notre dernier exercice lorsque nous avons prédit les images de la langue des signes. Rappelez-vous que dans ce cas, la forme finale de l'image doit être (1, 224, 224, 3).

Lorsque nous chargeons des modèles directement avec Keras, nous pouvons également profiter des méthodes [`preprocess_input`] (https://www.tensorflow.org/api_docs/python/tf/keras/applications/vgg16/preprocess_input). Ces méthodes, associées à un modèle spécifique, permettent aux utilisateurs de prétraiter leurs propres images pour qu'elles correspondent aux qualités des images sur lesquelles le modèle a été entraîné à l'origine.

In [ ]:
from tensorflow.keras.preprocessing import image as image_utils
from tensorflow.keras.applications.vgg16 import preprocess_input

def load_and_process_image(image_path):
    # Print image's original shape, for reference
    print('Original image shape: ', mpimg.imread(image_path).shape)
    
    # Load in the image with a target size of 224, 224
    image = image_utils.load_img(image_path, target_size=(224, 224))
    # Convert the image from a PIL format to a numpy array
    image = image_utils.img_to_array(image)
    # Add a dimension for number of images, in our case 1
    image = image.reshape(1,224,224,3)
    # Preprocess image to align with original ImageNet dataset
    image = preprocess_input(image)
    # Print image's shape after processing
    print('Processed image shape: ', image.shape)
    return image

In [ ]:
processed_image = load_and_process_image("data/doggy_door_images/brown_bear.jpg")

## Exercice : Faire une Prédiction

Maintenant que notre image est au bon format, nous pouvons la passer dans notre modèle et obtenir une prédiction. Nous nous attendons à obtenir un tableau de 1000 éléments, ce qui sera difficile à lire. Heureusement, les modèles chargés directement avec Keras disposent d'une autre méthode utile qui traduira ce tableau de prédiction sous une forme plus lisible. 

Remplissez la fonction suivante (dans les cases **FIXME**) pour implémenter la prédiction :

In [ ]:
from tensorflow.keras.applications.vgg16 import decode_predictions

def readable_prediction(image_path):
    # Show image
    FIXME
    # Load and pre-process image
    image = FIXME
    # Make predictions
    predictions = FIXME
    # Print predictions in readable form
    print('Predicted:', decode_predictions(predictions, top=3))

In [ ]:
#@title Solution
from tensorflow.keras.applications.vgg16 import decode_predictions

def readable_prediction(image_path):
    # Show image
    show_image(image_path)
    # Load and pre-process image
    image = load_and_process_image(image_path)
    # Make predictions
    predictions = model.predict(image)
    # Print predictions in readable form
    print('Predicted:', decode_predictions(predictions, top=3))

Essayez-le sur quelques animaux pour voir les résultats ! N'hésitez pas non plus à télécharger vos propres images et à les classer par catégorie pour voir comment cela fonctionne.

In [ ]:
readable_prediction("data/doggy_door_images/happy_dog.jpg")

In [ ]:
readable_prediction("data/doggy_door_images/brown_bear.jpg")

In [ ]:
readable_prediction("data/doggy_door_images/sleepy_cat.jpg")

## Exercice 2 : Seulement des Chiens

Maintenant que nous faisons des prédictions avec notre modèle, nous pouvons utiliser nos catégories pour ne laisser entrer et sortir que les chiens et garder les chats à l'intérieur. Rappelez-vous que les chiens correspondent aux catégories 151 à 268 et les chats aux catégories 281 à 285. La fonction [np.argmax](https://numpy.org/doc/stable/reference/generated/numpy.argmax.html) permet de déterminer quel élément du tableau de prédiction est la catégorie supérieure.

Remplissez les cases **FIXME**.

In [ ]:
import numpy as np

def doggy_door(image_path):
    show_image(image_path)
    image = load_and_process_image(image_path)
    preds = model.predict(image)
    if FIXME:
        print("Doggy come on in!")
    elif FIXME:
        print("Kitty stay inside!")
    else:
        print("You're not a dog! Stay outside!")

In [ ]:
#@title Solution

import numpy as np

def doggy_door(image_path):
    show_image(image_path)
    image = load_and_process_image(image_path)
    preds = model.predict(image)
    if 151 <= np.argmax(preds) <= 268:
        print("Doggy come on in!")
    elif 281 <= np.argmax(preds) <= 285:
        print("Kitty stay inside!")
    else:
        print("You're not a dog! Stay outside!")


## Validation

In [ ]:
doggy_door("data/doggy_door_images/brown_bear.jpg")

In [ ]:
doggy_door("data/doggy_door_images/happy_dog.jpg")

In [ ]:
doggy_door("data/doggy_door_images/sleepy_cat.jpg")

## Conclusion

Un travail remarquable ! En utilisant un puissant modèle pré-entraîné, nous avons créé une porte pour chien fonctionnelle en seulement quelques lignes de code. Nous espérons que vous êtes heureux de réaliser que vous pouvez tirer parti de l'apprentissage profond sans beaucoup de travail initial. Le plus intéressant est qu'au fur et à mesure que la communauté de l'apprentissage profond progresse, d'autres modèles seront disponibles pour que vous puissiez les utiliser dans vos propres projets.